In [6]:
import requests
from requests_oauthlib import OAuth1

import json

consumer_key = 'UCtyu2vxdx4M0XVyNc55JpbHZ'
consumer_secret = 'bbMoGKMviomJji2kNv5KjTm3jpShzZ1RS38gtBbrbC0U0mG5aB'
access_token = '902952608071114754-06K4FgECk6fk2IPeoA5JvN03GOUHJ6b'
access_secret = 'xWFHxIp6rq9MZVsWQnuvzeM8rAUSeiu1xvi514u3pzucn'

auth = OAuth1(consumer_key, consumer_secret, access_token, access_secret)
print(auth)

In [7]:
print(auth.client)

<Client client_key=UCtyu2vxdx4M0XVyNc55JpbHZ, client_secret=****, resource_owner_key=902952608071114754-06K4FgECk6fk2IPeoA5JvN03GOUHJ6b, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encoding=utf-8, decoding=utf-8, nonce=None, timestamp=None>


In [8]:
import urllib

url_2 = 'https://api.twitter.com/1.1/search/tweets.json'
params = {
    "count": 10, 
    "lang": 'en',
    "q": '%23nyu'
}

res = requests.get(url_2, auth=auth, params=params)

print(res, res.status_code, res.headers['content-type'])

tweets = res.json()

dic = {}
result = []

for num, tweet in enumerate(tweets['statuses']):
    dic={'username': tweet['user']['name'],
        'screen_name': tweet['user']['screen_name'],
        'text': tweet['text'],
        'location':tweet['user']['location']}
    result.append(dic)
#print(result)
#print(len(result))

<Response [200]> 200 application/json;charset=utf-8


In [12]:
##IF not mySQLdb is not installed

# PANOS: Please use SQL Alchemy as in 2/G3 notebook. 
# PANOS: Please use your actual MySQL database. The one on 127.0.0.1 is not usable
#import MySQLdb as mdb

#con = mdb.connect(host= '127.0.0.1',
#                 user = 'root',
#                 passwd = '',
#                 charset='utf8', use_unicode=True)
from sqlalchemy import create_engine
con = 'mysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = '35.237.137.195', 
    user = 'root',
    password = 'cNJ82xfyqM68LmA1',
    encoding = 'utf8mb4')
engine = create_engine(con)
con = engine.connect()

# Query to create a database
db_name = 'twit'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
#cursor = con.cursor()
#cursor.execute(create_db_query)
#cursor.close()
engine.execute(create_db_query)
engine.execute("USE {db}".format(db=db_name))

/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:508: Warning: (1007, "Can't create database 'twit'; database exists")
  cursor.execute(statement, parameters)


In [22]:
table_name = 'tweets'

# Create a table
# The {db} and {table} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (tweet_id int, 
                                name varchar(250), 
                                screen_name varchar(250), 
                                text varchar(250), 
                                location varchar(250), 
                                PRIMARY KEY(tweet_id)
                                )'''.format(db=db_name, table=table_name)
#CHANGE CHARSET TO utf8mb4
change_charset = '''ALTER TABLE {db}.{table} 
                    CHANGE text text VARCHAR(250), 
                    CHANGE name name VARCHAR(250),                    
                    CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;'''.format(db=db_name, table=table_name)

engine.execute(change_charset)    
engine.execute(create_table_query)

/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:508: Warning: (1050, "Table 'tweets' already exists")
  cursor.execute(statement, parameters)


In [23]:
query_template = '''INSERT INTO {db}.{table}(
                                            tweet_id,
                                            name, 
                                            screen_name, 
                                            text, 
                                            location) 
                    VALUES (%s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)

for num, entry in enumerate(result):
    tweet_id = num
    name = entry["username"]
    screenName = entry["screen_name"]
    text = entry["text"]
    location = entry["location"]
    
    query_parameters = (tweet_id, name, screenName, text, location)
    engine.execute(query_template, query_parameters)
